In [ ]:
import os, requests, pathlib
from mypackage import finder

data_dir = os.path.join(finder.get_git_root(), "data")
db_name = "Chinook.db"

url = "https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db"

db_path = pathlib.Path(os.path.join(data_dir, db_name))

if not db_path.exists():
    response = requests.get(url)
    if response.status_code == 200:
        db_path.write_bytes(response.content)
        print(f"Downloaded {db_path}")
    else:
        print(f"Failed to download file: {response.status_code}")
else:
    print(f"File {db_path} already exists.")

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")
print(f'Sample output: {db.run("SELECT * FROM Artist LIMIT 5;")}')

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    "llama3.2",
    model_provider="ollama",
    temperature=0,
    max_tokens=1024,
)

In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)  # Replace None with your LLM instance

tools = toolkit.get_tools()

for tool in tools:
    print(f"Tool name: {tool.name}, description: {tool.description}")

In [ ]:
system_prompt = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
""".format(
    dialect=db.dialect,
    top_k=5,
)

In [ ]:
from langchain.agents import create_agent
agent = create_agent(
    llm,
    tools,
    system_prompt=system_prompt
)

In [ ]:
from mypackage import userinput

user_query = userinput.get_user_input("Stelle deine Frage!", default="List the 10 longest tracks in the database.")


for step in agent.stream(
    {"messages": [{"role": "user", "content": user_query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()